In [1]:
import os
import pickle

from skimage.io import imread
from skimage.transform import resize
import numpy as np
from cuml.model_selection import train_test_split
from cuml.svm import SVC
from sklearn.metrics import accuracy_score


In [7]:
# prepare data
input_dir = 'brain_tumor_mris'
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

data = []
labels = []

for category_idx, category in enumerate(categories):
    for file in os.listdir(os.path.join(input_dir, category)):
        img_path = os.path.join(input_dir, category, file)
        img = imread(img_path, as_gray=True)
        img = resize(img, (512, 512), anti_aliasing=True)
        data.append(img.flatten())
        labels.append(category_idx)

data = np.asarray(data)
labels = np.asarray(labels)

In [ ]:
# train / test split
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

# train classifier
classifier = SVC()

# Here we would use GridSearchCV from cuml when it becomes available
# For now, let's assume we manually set hyperparameters
classifier.set_params(C=10, gamma=0.001)

classifier.fit(x_train, y_train)

# test performance
y_prediction = classifier.predict(x_test)

score = accuracy_score(y_prediction, y_test)

print('{}% of samples were correctly classified'.format(str(score * 100)))

pickle.dump(classifier, open('./mri-model-v3(dimension-increase-512x512).p', 'wb'))


/home/luis/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/cupy/_creation/from_data.py:88: PerformanceWarning: Using synchronous transfer as pinned memory (9797894144 bytes) could not be allocated. This generally occurs because of insufficient host memory. The original error was: cudaErrorMemoryAllocation: out of memory
  return _core.array(a, dtype, False, order, blocking=blocking)
/home/luis/miniconda3/envs/rapids-24.06/lib/python3.11/site-packages/cupy/_creation/from_data.py:88: PerformanceWarning: Using synchronous transfer as pinned memory (3917479936 bytes) could not be allocated. This generally occurs because of insufficient host memory. The original error was: cudaErrorMemoryAllocation: out of memory
  return _core.array(a, dtype, False, order, blocking=blocking)
